In [154]:
import numpy as np
import pandas as pd
import sklearn
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn import feature_extraction,model_selection
from sklearn.metrics import confusion_matrix
import glob
import os
import string
import tensorflow as tf
import keras as K
import zipfile
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# importing Word2Vec
from gensim.models import Word2Vec
from gensim.models import FastText

In [0]:
df=pd.read_csv('/content/hin_train.csv')

In [157]:
X=df[['Text','Sub-task B']]
X.head()

Text Sub-task B
0  Bollywood film dekhne ke samay logic ghar mein...       NGEN
1                                   Chutiya movie...       NGEN
2  Us jaat bnde ka khene ka matlab tha mar daluga...       NGEN
3  @Feminism Is CANCER *un feminist yeh sahi hai ...       NGEN
4       Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने       NGEN

In [0]:
df1=pd.read_csv('/content/hin_dev.csv')

In [159]:
Y=df1[['Text','Sub-task B']]
Y.head()

Text Sub-task B
0  bkl interviewers kuch jaada hi open minded bnt...       NGEN
1  Bhaiya shaadi mein zaroor aana movie ka plot j...       NGEN
2  Section 375 hai kya??? .... Ye to batate kam s...       NGEN
3  कबीर सिंह hit Hui इससे पता चलता है आजकल के लोग...       NGEN
4             Maine itni kam dislike kbhi nhii dekhi       NGEN

In [160]:
cols=['Sub-task B']
X[cols] = X[cols].apply(lambda x: pd.factorize(x)[0] + 0)
Y[cols] = Y[cols].apply(lambda x: pd.factorize(x)[0] + 0)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [161]:
X.head()

Text  Sub-task B
0  Bollywood film dekhne ke samay logic ghar mein...           0
1                                   Chutiya movie...           0
2  Us jaat bnde ka khene ka matlab tha mar daluga...           0
3  @Feminism Is CANCER *un feminist yeh sahi hai ...           0
4       Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने           0

In [162]:
Y.head()

Text  Sub-task B
0  bkl interviewers kuch jaada hi open minded bnt...           0
1  Bhaiya shaadi mein zaroor aana movie ka plot j...           0
2  Section 375 hai kya??? .... Ye to batate kam s...           0
3  कबीर सिंह hit Hui इससे पता चलता है आजकल के लोग...           0
4             Maine itni kam dislike kbhi nhii dekhi           0

In [163]:
print(X['Sub-task B'].value_counts())
print(Y['Sub-task B'].value_counts())



0    3323
1     661
Name: Sub-task B, dtype: int64
0    845
1    152
Name: Sub-task B, dtype: int64


In [164]:
X_train=X['Text']
X_train

0       Bollywood film dekhne ke samay logic ghar mein...
1                                        Chutiya movie...
2       Us jaat bnde ka khene ka matlab tha mar daluga...
3       @Feminism Is CANCER *un feminist yeh sahi hai ...
4            Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने
                              ...                        
3979    Asexual h.. bisexual... homosexual... bhai ase...
3980    Video pura dekne ke pahile hi mai bhai ke vide...
3981                     konsa place hai bhai ...nam bolo
3982                             Kuch zada hi likh diya 🙄
3983    scary tube par kya woh karna sahi tha? Pucha k...
Name: Text, Length: 3984, dtype: object

In [165]:
y_train=X['Sub-task B']
y_train

0       0
1       0
2       0
3       0
4       0
       ..
3979    0
3980    0
3981    0
3982    0
3983    0
Name: Sub-task B, Length: 3984, dtype: int64

In [166]:
X_test=Y['Text']
X_test

0      bkl interviewers kuch jaada hi open minded bnt...
1      Bhaiya shaadi mein zaroor aana movie ka plot j...
2      Section 375 hai kya??? .... Ye to batate kam s...
3      कबीर सिंह hit Hui इससे पता चलता है आजकल के लोग...
4                 Maine itni kam dislike kbhi nhii dekhi
                             ...                        
992                Are bhai konsi duniya mai rehate ho😂😂
993                                         Sahi to bola
994                              Or tum Kay Kar raya ho?
995    Ye bhaat sahi hain par zyada dhin thine ga nah...
996                 Islye maine Kabaad singh nai dekha..
Name: Text, Length: 997, dtype: object

In [167]:
y_test=Y['Sub-task B']
y_test

0      0
1      0
2      0
3      0
4      0
      ..
992    0
993    0
994    0
995    0
996    0
Name: Sub-task B, Length: 997, dtype: int64

In [168]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3984,), (997,), (3984,), (997,))

In [0]:
def create_tokens(lines):
  #stop_words = set(nltk.corpus.stopwords.words('english'))
  t_line=lines.apply(lambda x: nltk.word_tokenize(x))
  #t_line=t_line.apply(lambda x: [w for w in x if not w in stop_words])
  return t_line

In [170]:
X_train=create_tokens(X_train)
print(X_train)
tokenizer= Tokenizer()
tokenizer.fit_on_texts(X_train)
tokenizer


0       [Bollywood, film, dekhne, ke, samay, logic, gh...
1                                   [Chutiya, movie, ...]
2       [Us, jaat, bnde, ka, khene, ka, matlab, tha, m...
3       [@, Feminism, Is, CANCER, *un, feminist, yeh, ...
4       [Amrit, Anand, अब, तो, जुड़े, ही, है, उनको, बोल...
                              ...                        
3979    [Asexual, h.., bisexual, ..., homosexual, ...,...
3980    [Video, pura, dekne, ke, pahile, hi, mai, bhai...
3981            [konsa, place, hai, bhai, ..., nam, bolo]
3982                      [Kuch, zada, hi, likh, diya, 🙄]
3983    [scary, tube, par, kya, woh, karna, sahi, tha,...
Name: Text, Length: 3984, dtype: object


In [0]:
length=np.average(X_train.apply(lambda x:len(x)))
length=20

In [0]:
def encode_text(tokenizer, lines, length):
  encoded = tokenizer.texts_to_sequences(lines)
  padded = pad_sequences(encoded, maxlen=length, padding='post')
  return padded

In [0]:
word_index=tokenizer.word_index

In [174]:
X_test=create_tokens(X_test)
print(X_test)

0      [bkl, interviewers, kuch, jaada, hi, open, min...
1      [Bhaiya, shaadi, mein, zaroor, aana, movie, ka...
2      [Section, 375, hai, kya, ?, ?, ?, ..., ., Ye, ...
3      [कबीर, सिंह, hit, Hui, इससे, पता, चलता, है, आज...
4         [Maine, itni, kam, dislike, kbhi, nhii, dekhi]
                             ...                        
992        [Are, bhai, konsi, duniya, mai, rehate, ho😂😂]
993                                     [Sahi, to, bola]
994                     [Or, tum, Kay, Kar, raya, ho, ?]
995    [Ye, bhaat, sahi, hain, par, zyada, dhin, thin...
996          [Islye, maine, Kabaad, singh, nai, dekha..]
Name: Text, Length: 997, dtype: object


In [175]:
X_train=encode_text(tokenizer,X_train,length)
X_train
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[   85,    59,   175, ...,     0,     0,     0],
       [   72,    13,     2, ...,     0,     0,     0],
       [ 5148,   857,    20, ...,     1,  5152,  5153],
       ...,
       [ 1945,  3236,     1, ...,     0,     0,     0],
       [   49,  4115,    18, ...,     0,     0,     0],
       [   17,    83,   445, ...,  1464,  3098, 14125]], dtype=int32)

In [176]:
X_test=encode_text(tokenizer,X_test,length)
X_test
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[  18, 1873, 3546, ...,  705,  763,   33],
       [ 342, 1484,   44, ...,    0,    0,    0],
       [ 572,  492,    1, ...,    0,    0,    0],
       ...,
       [  43,   78,  898, ...,    0,    0,    0],
       [  16, 2383,   35, ...,    0,    0,    0],
       [7446,  173,   48, ...,    0,    0,    0]], dtype=int32)

In [177]:
from keras.utils.np_utils import to_categorical
y_test = to_categorical(np.asarray(y_test))
y_train = to_categorical(np.asarray(y_train))
print(y_test)
print(y_train)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [77]:
# print(len(X_train))
# print(len(X_test))
# print(len(y_train))
# print(len(y_test))
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3984, 685), (3984, 685), (3984, 2), (3984, 2))

In [0]:
# Creating embedding matrix
# Using 
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/Copy of crawl-300d-2M.vec.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [36]:
#load pre-trained embedding:
embeddings_index = {}
f = open('/content/crawl-300d-2M.vec', encoding="utf8")
for line in f:
    values = line.split()#This splits each values in a  line with spaces
    word = values[0] # This store the first values which is the word in english
    coefs = np.asarray(values[1:], dtype='float32') # This converts all splitted values from 1 to last(expect first word) into an array
    embeddings_index[word] = coefs # english word = array of numerical values
f.close()
print('Total %s word vectors in crawl-300-2M.' % len(embeddings_index))

Total 1999996 word vectors in crawl-300-2M.


In [37]:
print((X_train))
print((X_test))
print((y_train))
print((y_test))

[[  80   54  160 ...    0    0    0]
 [  67   12    2 ...    0    0    0]
 [ 331  614 5096 ...    0    0    0]
 ...
 [1897 3187    1 ...    0    0    0]
 [  44 4067   16 ...    0    0    0]
 [4516 2224   60 ...    0    0    0]]
[[  80   54  160 ...    0    0    0]
 [  67   12    2 ...    0    0    0]
 [ 331  614 5096 ...    0    0    0]
 ...
 [1897 3187    1 ...    0    0    0]
 [  44 4067   16 ...    0    0    0]
 [4516 2224   60 ...    0    0    0]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [0]:
# Initialize the weight for each word for the embedding layer. Words not present
# in the dictionary initialized by random value 
EMBEDDING_DIM = 300
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
#embedding_matrix = (16433, 100) NOTE: It is from uniques words in our tweets
for word, i in word_index.items(): # Selecting word from our tweet unique words list for changing with the values from glove
    embedding_vector = embeddings_index.get(word) # Finding our tweet words in glove and taking its array of 100 values
    if embedding_vector is not None: 
#words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [180]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.utils.vis_utils import plot_model
model = Sequential()
model.add(Embedding(input_dim = 14126, output_dim = 300, weights=[embedding_matrix], input_length=length))
model.add(Conv1D(filters = 128,kernel_size=3, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 20, 300)           4237800   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 18, 128)           115328    
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 3, 128)            0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               98560     
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 514       
Total params: 4,452,202
Trainable params: 4,452,202
Non-trainable params: 0
____________________________________________

In [181]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),epochs=100,batch_size=100, verbose=2)

Train on 3984 samples, validate on 997 samples
Epoch 1/100
 - 1s - loss: 0.4569 - acc: 0.8338 - val_loss: 0.4336 - val_acc: 0.8475
Epoch 2/100
 - 0s - loss: 0.4489 - acc: 0.8341 - val_loss: 0.4329 - val_acc: 0.8475
Epoch 3/100
 - 0s - loss: 0.4416 - acc: 0.8338 - val_loss: 0.4303 - val_acc: 0.8475
Epoch 4/100
 - 0s - loss: 0.4388 - acc: 0.8341 - val_loss: 0.4280 - val_acc: 0.8475
Epoch 5/100
 - 0s - loss: 0.4340 - acc: 0.8341 - val_loss: 0.4266 - val_acc: 0.8475
Epoch 6/100
 - 0s - loss: 0.4286 - acc: 0.8341 - val_loss: 0.4272 - val_acc: 0.8475
Epoch 7/100
 - 0s - loss: 0.4256 - acc: 0.8341 - val_loss: 0.4248 - val_acc: 0.8475
Epoch 8/100
 - 0s - loss: 0.4210 - acc: 0.8341 - val_loss: 0.4271 - val_acc: 0.8475
Epoch 9/100
 - 0s - loss: 0.4175 - acc: 0.8346 - val_loss: 0.4248 - val_acc: 0.8465
Epoch 10/100
 - 0s - loss: 0.4142 - acc: 0.8343 - val_loss: 0.4292 - val_acc: 0.8465
Epoch 11/100
 - 0s - loss: 0.4109 - acc: 0.8348 - val_loss: 0.4206 - val_acc: 0.8475
Epoch 12/100
 - 0s - loss: 

In [182]:
predictions_test = model.predict(X_test)

predictions_test

array([[9.6119761e-01, 3.8802415e-02],
       [9.9901748e-01, 9.8247861e-04],
       [9.8126298e-01, 1.8737033e-02],
       ...,
       [9.9364167e-01, 6.3582924e-03],
       [9.9821484e-01, 1.7851797e-03],
       [9.9834514e-01, 1.6548367e-03]], dtype=float32)

In [0]:
pred_test=np.zeros_like(predictions_test)
pred_test[np.arange(len(predictions_test)),predictions_test.argmax(1)]=1

In [187]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred_test))

              precision    recall  f1-score   support

           0       0.87      0.96      0.91       845
           1       0.47      0.18      0.26       152

   micro avg       0.84      0.84      0.84       997
   macro avg       0.67      0.57      0.59       997
weighted avg       0.81      0.84      0.81       997
 samples avg       0.84      0.84      0.84       997



In [0]:
model.save('HindiTaskB.h5')